In [122]:
import findspark
import numpy as np
from pyspark import StorageLevel

findspark.init()
findspark.find()

'C:\\spark-3.0.1-bin-hadoop2.7'

In [123]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [124]:
%%time

#initialization of spark session
sk = SparkSession.builder.appName("RDD_NBA_stats").getOrCreate()

#load the datasets
df = sk.read.format("csv").option("header", "true").load("NBA_Data/rs_10-20.csv")



Wall time: 217 ms


In [125]:
df.persist(StorageLevel.MEMORY_ONLY)
df.is_cached

True

In [126]:
df

DataFrame[ID: string, TEAM: string, GP: string, W: string, L: string, WIN%: string, MIN: string, PTS: string, FGM: string, FGA: string, FG%: string, 3PM: string, 3PA: string, 3P%: string, FTM: string, FTA: string, FT%: string, OREB: string, DREB: string, REB: string, AST: string, TOV: string, STL: string, BLK: string, BLKA: string, PF: string, PFD: string, +/-: string, SEASN: string]

In [127]:
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- TEAM: string (nullable = true)
 |-- GP: string (nullable = true)
 |-- W: string (nullable = true)
 |-- L: string (nullable = true)
 |-- WIN%: string (nullable = true)
 |-- MIN: string (nullable = true)
 |-- PTS: string (nullable = true)
 |-- FGM: string (nullable = true)
 |-- FGA: string (nullable = true)
 |-- FG%: string (nullable = true)
 |-- 3PM: string (nullable = true)
 |-- 3PA: string (nullable = true)
 |-- 3P%: string (nullable = true)
 |-- FTM: string (nullable = true)
 |-- FTA: string (nullable = true)
 |-- FT%: string (nullable = true)
 |-- OREB: string (nullable = true)
 |-- DREB: string (nullable = true)
 |-- REB: string (nullable = true)
 |-- AST: string (nullable = true)
 |-- TOV: string (nullable = true)
 |-- STL: string (nullable = true)
 |-- BLK: string (nullable = true)
 |-- BLKA: string (nullable = true)
 |-- PF: string (nullable = true)
 |-- PFD: string (nullable = true)
 |-- +/-: string (nullable = true)
 |-- SEASN: strin

In [129]:
%%time
# Query1 
# witch team has the most field GOAL precentage? (and its win%)
# first gather all the data 

rdd = df.rdd
rdd = rdd.map(lambda x: (x[1],x[2], x[3]))\
.sortBy(lambda x: x[1], ascending=False)
rdd.take(25)
    


Wall time: 654 ms


[('Chicago Bulls', '82', '62'),
 ('San Antonio Spurs', '82', '61'),
 ('Miami Heat', '82', '58'),
 ('Dallas Mavericks', '82', '57'),
 ('Los Angeles Lakers', '82', '57'),
 ('Boston Celtics', '82', '56'),
 ('Oklahoma City Thunder', '82', '55'),
 ('Orlando Magic', '82', '52'),
 ('Denver Nuggets', '82', '50'),
 ('Portland Trail Blazers', '82', '48'),
 ('Memphis Grizzlies', '82', '46'),
 ('New Orleans Hornets', '82', '46'),
 ('Atlanta Hawks', '82', '44'),
 ('Houston Rockets', '82', '43'),
 ('New York Knicks', '82', '42'),
 ('Philadelphia 76ers', '82', '41'),
 ('Phoenix Suns', '82', '40'),
 ('Utah Jazz', '82', '39'),
 ('Indiana Pacers', '82', '37'),
 ('Golden State Warriors', '82', '36'),
 ('Milwaukee Bucks', '82', '35'),
 ('Charlotte Bobcats', '82', '34'),
 ('Los Angeles Clippers', '82', '32'),
 ('Detroit Pistons', '82', '30'),
 ('New Jersey Nets', '82', '24')]

In [130]:
%%time
# Query 2 reduce them
rdd = rdd.map(lambda x: (x[0] , [float(x[1]) , float(x[2]) , 1]))\
.reduceByKey(lambda x,y : [x[0] + y[0] , x[1] + y[1],x[2] + y[2]])\
.map(lambda x: [x[0],x[1][0] /x[1][2] ,x[1][1]])\
.sortBy(lambda x: x[1], ascending=False)
rdd.collect()

Wall time: 1.46 s


[['Brooklyn Nets', 80.75, 277.0],
 ['New Orleans Pelicans', 80.57142857142857, 254.0],
 ['LA Clippers', 80.0, 243.0],
 ['Dallas Mavericks', 79.7, 408.0],
 ['Portland Trail Blazers', 79.6, 436.0],
 ['Miami Heat', 79.5, 477.0],
 ['Orlando Magic', 79.5, 321.0],
 ['Denver Nuggets', 79.5, 430.0],
 ['Memphis Grizzlies', 79.5, 422.0],
 ['Philadelphia 76ers', 79.5, 331.0],
 ['Phoenix Suns', 79.5, 306.0],
 ['Milwaukee Bucks', 79.5, 395.0],
 ['Oklahoma City Thunder', 79.4, 509.0],
 ['Houston Rockets', 79.4, 490.0],
 ['Utah Jazz', 79.4, 414.0],
 ['Indiana Pacers', 79.4, 450.0],
 ['Sacramento Kings', 79.4, 293.0],
 ['Washington Wizards', 79.4, 352.0],
 ['Toronto Raptors', 79.4, 453.0],
 ['San Antonio Spurs', 79.3, 541.0],
 ['Los Angeles Lakers', 79.3, 358.0],
 ['Boston Celtics', 79.3, 454.0],
 ['Charlotte Hornets', 79.16666666666667, 215.0],
 ['Atlanta Hawks', 78.9, 390.0],
 ['New York Knicks', 78.8, 316.0],
 ['Los Angeles Clippers', 78.8, 241.0],
 ['Detroit Pistons', 78.8, 326.0],
 ['Chicago Bull

In [131]:
# Query 3
# 1. Can be (the best most field GOAL) guarantee to be (the best winner)?
max_field_goal_item = rdd.sortBy(lambda x: x[1], ascending=False).take(1)

max_field_goal , win_percent = max_field_goal_item[0][1] , max_field_goal_item[0][2]
print("max field goal : " + str(max_field_goal) + "\n"
      + "its max win percent : " + str(win_percent))

q1rdd = rdd.filter(lambda x: x[2] >= win_percent)\
.sortBy(lambda x: x[2], ascending=False)


if(len(q1rdd.collect()) >0 ):
    print("\nThere are some teams which their field goal is less than max field goal\nSo, it doesn't guarantee\n")
else:
     print("\nIt guarantees\n")
q1rdd.collect()

max field goal : 80.75
its max win percent : 277.0

There are some teams which their field goal is less than max field goal
So, it doesn't guarantee



[['San Antonio Spurs', 79.3, 541.0],
 ['Oklahoma City Thunder', 79.4, 509.0],
 ['Golden State Warriors', 78.7, 494.0],
 ['Houston Rockets', 79.4, 490.0],
 ['Miami Heat', 79.5, 477.0],
 ['Boston Celtics', 79.3, 454.0],
 ['Toronto Raptors', 79.4, 453.0],
 ['Indiana Pacers', 79.4, 450.0],
 ['Portland Trail Blazers', 79.6, 436.0],
 ['Denver Nuggets', 79.5, 430.0],
 ['Memphis Grizzlies', 79.5, 422.0],
 ['Utah Jazz', 79.4, 414.0],
 ['Chicago Bulls', 78.7, 409.0],
 ['Dallas Mavericks', 79.7, 408.0],
 ['Milwaukee Bucks', 79.5, 395.0],
 ['Atlanta Hawks', 78.9, 390.0],
 ['Los Angeles Lakers', 79.3, 358.0],
 ['Washington Wizards', 79.4, 352.0],
 ['Cleveland Cavaliers', 78.7, 346.0],
 ['Philadelphia 76ers', 79.5, 331.0],
 ['Detroit Pistons', 78.8, 326.0],
 ['Orlando Magic', 79.5, 321.0],
 ['New York Knicks', 78.8, 316.0],
 ['Phoenix Suns', 79.5, 306.0],
 ['Sacramento Kings', 79.4, 293.0],
 ['Minnesota Timberwolves', 78.6, 292.0],
 ['Brooklyn Nets', 80.75, 277.0]]

In [132]:
%%time
# Q4 how the FGM (Field Goals Made) percentage influences the winning of a title?
rdd = df.rdd.cache()
rdd = rdd.map(lambda x: (x[1],x[8])).sortBy(lambda x: x[1], ascending=False)
rdd.take(10)

Wall time: 722 ms


[('Golden State Warriors', '44'),
 ('New Orleans Pelicans', '43.7'),
 ('Milwaukee Bucks', '43.4'),
 ('Milwaukee Bucks', '43.3'),
 ('Sacramento Kings', '43.2'),
 ('Golden State Warriors', '43.1'),
 ('Golden State Warriors', '42.8'),
 ('New Orleans Pelicans', '42.7'),
 ('Oklahoma City Thunder', '42.6'),
 ('Los Angeles Lakers', '42.6')]

In [134]:
%%time
# Query 5
# group by data (Win)
rdd = df.rdd.cache()
rdd = rdd.map(lambda x: (x[0],x[1],x[2],x[5],x[7],x[28]))\
.groupBy(lambda x: x[3])\
.mapValues(list)

rdd.collect()

Wall time: 1.48 s


[('0.756',
  [('1', 'Chicago Bulls', '82', '0.756', '98.6', "'10-11'"),
   ('1', 'San Antonio Spurs', '82', '0.756', '105.4', "'13-14'")]),
 ('0.744',
  [('2', 'San Antonio Spurs', '82', '0.744', '103.7', "'10-11'"),
   ('2', 'San Antonio Spurs', '82', '0.744', '105.3', "'16-17'")]),
 ('0.707',
  [('3', 'Miami Heat', '82', '0.707', '102.1', "'10-11'"),
   ('3', 'San Antonio Spurs', '82', '0.707', '103', "'12-13'"),
   ('3', 'Golden State Warriors', '82', '0.707', '113.5', "'17-18'"),
   ('2', 'Toronto Raptors', '82', '0.707', '114.4', "'18-19'")]),
 ('0.695',
  [('4', 'Dallas Mavericks', '82', '0.695', '100.2', "'10-11'"),
   ('4', 'Los Angeles Lakers', '82', '0.695', '101.5', "'10-11'"),
   ('4', 'Denver Nuggets', '82', '0.695', '106.1', "'12-13'"),
   ('3', 'Los Angeles Clippers', '82', '0.695', '107.9', "'13-14'"),
   ('3', 'Cleveland Cavaliers', '82', '0.695', '104.3', "'15-16'"),
   ('3', 'Golden State Warriors', '82', '0.695', '117.7', "'18-19'")]),
 ('0.683',
  [('6', 'Boston Ce